In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.io import loadmat
import seaborn as sns

%matplotlib inline
%reload_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize'] = 10, 5

# Range-only data for localization 

WiFi: http://www.robesafe.es/repository/UAHWiFiDataset/

Landmower: https://github.com/gtrll/gpslam/tree/master/matlab/data

Download .mat files and save in folder ./datasets/


File description (from WiFi website, seems to be similar for Landmower): 
```
GT: Groundtruth path from Laser Scan matching
Time (sec) 	X_pose (m) 	Y_pose (m) 	Heading (rad)

DR: Odometry Input (delta distance traveled and delta heading change)
Time (sec) 	Delta Dist. Trav. (m) 	Delta Heading (rad)

DR_PROC: Dead Reckoned Path from Odometry
Time (sec) 	X_pose (m) 	Y_pose (m) 	Heading (rad)

TL: Surveyed Beacon Locations
Time (sec) 	X_pose (m) 	Y_pose (m)
*NOTE by Frederike: above is probably ID instead of time.

TD:
Time (sec) 	Sender / Antenna ID 	Receiver Node ID 	Range (m)
```

In [ ]:
# Need to give different systems a name.
gt_system_id = "GT"
gt_anchor_id = "GT"
range_system_id = "Range"

t_window = 1 # Time window to use for ground truth interpolation (see add_gt_raw).

filename = 'datasets/uah1.mat'; # fingers
#filename = 'datasets/uah2.mat'; # square
#filename = 'datasets/Plaza1.mat'; # zig zag
#filename = 'datasets/Plaza2.mat'; # triangle

max_time = 1000 # maximum time to analyze.

result_dict = loadmat(filename)
print('keys:', result_dict.keys())

In [ ]:
from evaluate_dataset import format_anchors_df, format_data_df

anchor_data = result_dict['TL']
anchors_df = pd.DataFrame(columns=['anchor_id', 'system_id', 'px', 'py', 'pz'])
anchor_ids = np.unique(anchor_data[:, 0])
for i, anchor_id in enumerate(anchor_ids):
    anchors_df.loc[i, 'anchor_id'] = anchor_id 
    anchors_df.loc[i, 'system_id'] = range_system_id
    
    # it is weird that there is more than one value for each anchor, it looks
    # like this was a bug in the dataset. we make sure they are all
    # the same and pick the first.
    px_values = np.unique(anchor_data[anchor_data[:, 0]==anchor_id, 1])
    py_values = np.unique(anchor_data[anchor_data[:, 0]==anchor_id, 2])
    assert len(px_values) == 1
    assert len(py_values) == 1
    anchors_df.loc[i, 'px'] = px_values[0]
    anchors_df.loc[i, 'py'] = py_values[0]
    
anchors_df = format_anchors_df(anchors_df, range_system_id=range_system_id, 
                               gt_system_id=gt_system_id)
print('anchors: \n', anchors_df)

range_df = pd.DataFrame(columns=['timestamp', 'px', 'py', 'pz', 'distance', 
                            'system_id', 'anchor_id'])
range_df.loc[:, 'distance'] = result_dict['TD'][:, 3]
range_df.loc[:, 'timestamp'] = result_dict['TD'][:, 0]
range_df.loc[:, 'anchor_id'] = result_dict['TD'][:, 2]
range_df.loc[:, 'system_id'] = range_system_id 

gt_df = pd.DataFrame(columns=range_df.columns)
gt_df.loc[:, 'px'] = result_dict['GT'][:, 1]
gt_df.loc[:, 'py'] = result_dict['GT'][:, 2]
gt_df.loc[:, 'timestamp'] = result_dict['GT'][:, 0]
gt_df.loc[:, 'anchor_id'] = gt_anchor_id
gt_df.loc[:, 'system_id'] = gt_system_id

full_df = pd.concat([range_df, gt_df], ignore_index=True)
full_df.sort_values('timestamp', inplace=True)
full_df.reset_index(drop=True, inplace=True)
full_df.loc[:, 'timestamp'] = full_df.timestamp -full_df.timestamp.min()
print('time going from {:.1f} to {:.1f}'.format(full_df.timestamp.min(), full_df.timestamp.max()))
dropping = len(full_df[full_df.timestamp > max_time])
print('warning: dropping {}/{}'.format(dropping, len(full_df)))
full_df = full_df[full_df.timestamp <= max_time]
full_df = format_data_df(full_df, anchors_df, gt_system_id=gt_system_id, 
                         range_system_id=range_system_id)

In [ ]:
fig, axs = plt.subplots(1, 2)
sns.scatterplot(data=full_df, x='px', y='py', hue='timestamp', linewidth=0.0, 
                ax=axs[0])
sns.scatterplot(data=full_df, x='timestamp', y='px', hue='timestamp', linewidth=0.0, 
                 ax=axs[1])

In [ ]:
from evaluate_dataset import add_gt_raw
full_df = add_gt_raw(full_df, t_window=t_window, gt_system_id=gt_system_id)

In [ ]:
fig, axs = plt.subplots(1, 2)
range_df = full_df[full_df.system_id==range_system_id]
sns.scatterplot(data=range_df, x='px', y='py', hue='timestamp', linewidth=0.0, 
                ax=axs[0])
sns.scatterplot(data=anchors_df, x='px', y='py', linewidth=0.0,  ax=axs[0], color='red')
sns.scatterplot(data=range_df, x='timestamp', y='px', hue='timestamp', linewidth=0.0, 
                 ax=axs[1])

In [ ]:
from evaluate_dataset import apply_distance_gt
full_df.loc[:, "distance_gt"] = full_df.apply(lambda row: apply_distance_gt(row, anchors_df, gt_system_id=gt_system_id), axis=1)

In [ ]:
range_ids = full_df[full_df.system_id==range_system_id].anchor_id.unique()
fig, axs = plt.subplots(len(range_ids), sharex=True)
fig.set_size_inches(10, 10)
for i, anchor_id in enumerate(sorted(range_ids)):
    this_df = full_df[full_df.anchor_id==anchor_id]
    axs[i].scatter(this_df.timestamp, this_df.distance, color='red', label='measured distance')
    axs[i].scatter(this_df.timestamp, this_df.distance_gt, color='green', label='real distance')
    axs[i].legend(loc='upper right')
    axs[i].set_title('anchor {}'.format(anchor_id))
    axs[i].set_ylabel('distance [m]')
axs[i].set_xlabel('time [s]')

In [ ]:
from evaluate_dataset import get_length, compute_distance_matrix
from trajectory import Trajectory

dimension=2

# We want to find out the times of distance measurements, 
# and by how much we have moved.
range_df = full_df[full_df.system_id==range_system_id]
times = range_df.timestamp.unique()

# We need to translate the times to "trajectory space"
lengths = get_length(range_df)
distances = np.cumsum(lengths)
traj = Trajectory(n_complexity=5, dim=dimension, model='bandlimited', 
                  period=1000, full_period=True)
times_corr,*_ = traj.get_times_from_distances(arbitrary_distances=distances)
basis = traj.get_basis(times=times_corr)


chosen_distance = 'distance'
anchor_names = None # use all anchors.
D = compute_distance_matrix(full_df, anchors_df, 
                            anchor_names, times, chosen_distance)

plt.figure()
plt.matshow(D[:10, :])
plt.title('First 10 rows of D matrix')